# Лабораторная работа 3, студент Устинов Денис Александрович М8О-406Б-21

## 1. Выбор начальных условий

### a. Набор данных для задачи классификации

Задача: Определение типа активности человека на основе данных с носимых устройств

Набор данных: Human Activity Recognition Dataset (HAR) (https://archive.ics.uci.edu/dataset/240/human+activity+recognition+using+smartphones), где данные получены с носимых датчиков и классифицируются в такие категории, как ходьба, сидение, подъем по лестнице и т. д.

Обоснование выбора: HAR представляет практическую задачу для классификации с несколькими классами и используется в задачах мониторинга здоровья и физической активности. Задача распознавания активности с помощью KNN полезна и практична, так как такой алгоритм достаточно прост для реализации в системах с носимыми устройствами.

### b. Набор данных для задачи регрессии

Задача: Прогнозирование уровня выбросов CO2 автомобилей

Набор данных: CO2 Emission by Vehicles (https://www.kaggle.com/datasets/debajyotipodder/co2-emission-by-vehicles), включающий параметры автомобиля (тип двигателя, вес, объем двигателя) и уровень выбросов CO2.

Обоснование выбора: Прогнозирование выбросов позволяет оценивать экологическое воздействие автомобилей на основе их характеристик, что является важной задачей в экологии и инженерии. Такой набор данных также подходит для KNN, так как близость характеристик автомобилей позволяет строить точные прогнозы выбросов.

### c. Выбор метрик качества

Для задачи классификации:

1) Accuracy (Точность): доля правильно классифицированных объектов. Хорошо подходит для задачи с равномерным распределением классов.
2) Precision: можно интерпретировать как долю объектов, названных классификатором положительными и при этом действительно являющимися положительными
3) Recall: показывает, какую долю объектов положительного класса из всех объектов положительного класса нашел алгоритм.
2) F1-score: среднее гармоническое между точностью (precision) и полнотой (recall), особенно полезно, если классы несбалансированы, так как учитывает ложноположительные и ложноотрицательные предсказания.

Для задачи регрессии:

1) Mean Absolute Error (MAE): средняя абсолютная ошибка, показывает среднее отклонение предсказаний от фактических значений и хорошо интерпретируется.
2) Mean Squared Error (MSE): средняя квадратичная ошибка, показывает более крупные ошибки больше чем MAE, что делает его полезным для выявления больших отклонений в прогнозах.

## 2. Создание бейзлайна и оценка качества

### a. Обучить модели из sklearn (для классификации) для выбранных наборов данных и оценить качество моделей (для классификации) по выбранным метрикам на выбранных наборах данных

Импорт библиотек

In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import sklearn
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier


#### Предобработаем данные

Получение и очистка данных

In [9]:
features = list()
with open('UCI HAR Dataset/features.txt') as f:
    features = [line.split()[1] for line in f.readlines()]

Дубликаты:

In [10]:
seen = set()
uniq_features = []
for idx, x in enumerate(features):
    if x not in seen:
        uniq_features.append(x)
        seen.add(x)
    elif x + 'n' not in seen:
        uniq_features.append(x + 'n')
        seen.add(x + 'n')
    else:
        uniq_features.append(x + 'nn')
        seen.add(x + 'nn')
len(uniq_features)

561

Подготовим обучающую выборку

In [11]:
def map_func(arg):
    if arg == 1: 
        return 'WALKING'
    if arg == 2:
        return 'WALKING_UPSTAIRS'
    if arg == 3:
        return 'WALKING_DOWNSTAIRS'
    if arg == 4:
        return 'SITTING'
    if arg == 5:
        return 'STANDING'
    if arg == 6:
        return 'LAYING'

In [12]:
X_train = pd.read_csv('UCI HAR Dataset/train/X_train.txt', delim_whitespace=True, header=None, names=uniq_features)
X_train['subject'] = pd.read_csv('UCI HAR Dataset/train/subject_train.txt', header=None)

y_train = pd.read_csv('UCI HAR Dataset/train/y_train.txt', names=['Activity'])
y_train_labels = y_train.map(map_func)

train = X_train
train['Activity'] = y_train
train['ActivityName'] = y_train_labels
train.sample()

/var/folders/kc/ml55mx4s5fl9fqwbt_5gpdz49kkcmp/T/ipykernel_74992/3107987138.py:1: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  X_train = pd.read_csv('UCI HAR Dataset/train/X_train.txt', delim_whitespace=True, header=None, names=uniq_features)


,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",subject,Activity,ActivityName
4311,0.262094,-0.027932,-0.086889,-0.192912,0.054534,-0.148468,-0.218344,0.057878,-0.145697,0.038973,...,0.168868,0.10496,0.618942,-0.515631,-0.60454,0.333589,0.156738,21,1,WALKING


Подготовим тестовую выборку

In [13]:
X_test = pd.read_csv('UCI HAR Dataset/test/X_test.txt', delim_whitespace=True, header=None, names=uniq_features)
X_test['subject'] = pd.read_csv('UCI HAR Dataset/test/subject_test.txt', header=None)

y_test = pd.read_csv('UCI HAR Dataset/test/y_test.txt', names=['Activity'])
y_test_labels = y_test.map(map_func)

test = X_test
test['Activity'] = y_test
test['ActivityName'] = y_test_labels
test.sample()

/var/folders/kc/ml55mx4s5fl9fqwbt_5gpdz49kkcmp/T/ipykernel_74992/2096605161.py:1: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  X_test = pd.read_csv('UCI HAR Dataset/test/X_test.txt', delim_whitespace=True, header=None, names=uniq_features)


,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",subject,Activity,ActivityName
2505,0.333812,-0.031515,-0.09217,-0.170736,0.730912,-0.290425,-0.215491,0.659842,-0.266653,0.024706,...,-0.745798,0.239962,0.623118,-0.958996,-0.627935,0.354491,-0.000642,20,1,WALKING


Поменяем названия функций

In [14]:
columns = train.columns

columns = columns.str.replace('[()]','')
columns = columns.str.replace('[-]', '')
columns = columns.str.replace('[,]','')

train.columns = columns
test.columns = columns

test.columns

Index(['tBodyAcc-mean()-X', 'tBodyAcc-mean()-Y', 'tBodyAcc-mean()-Z',
       'tBodyAcc-std()-X', 'tBodyAcc-std()-Y', 'tBodyAcc-std()-Z',
       'tBodyAcc-mad()-X', 'tBodyAcc-mad()-Y', 'tBodyAcc-mad()-Z',
       'tBodyAcc-max()-X',
       ...
       'angle(tBodyAccMean,gravity)', 'angle(tBodyAccJerkMean),gravityMean)',
       'angle(tBodyGyroMean,gravityMean)',
       'angle(tBodyGyroJerkMean,gravityMean)', 'angle(X,gravityMean)',
       'angle(Y,gravityMean)', 'angle(Z,gravityMean)', 'subject', 'Activity',
       'ActivityName'],
      dtype='object', length=564)

Сохраним выборки как csv

In [15]:
train.to_csv('data/train.csv', index=False)
test.to_csv('data/test.csv', index=False)

In [16]:
def read_data(file):
    data = pd.read_csv(file)
    
    data = sklearn.utils.shuffle(data)
    
    X_data = data.drop(['subject', 'Activity', 'ActivityName'], axis=1)
    y_data = data.ActivityName
    
    return np.array(X_data), np.array(y_data)

### a. Обучить модели с решающим деревом из sklearn (для классификации) для выбранных наборов данных и оценить качество моделей (для классификации) по выбранным метрикам на выбранных наборах данных

In [21]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

train_X, train_y = read_data('data/train.csv')
test_X, test_y = read_data('data/test.csv')

dt_classifier = DecisionTreeClassifier(criterion='entropy', max_depth=None, random_state=42)

dt_classifier.fit(train_X, train_y)

y_pred = dt_classifier.predict(test_X)

accuracy = accuracy_score(test_y, y_pred)
recall = recall_score(test_y, y_pred, average='weighted')
precision = precision_score(test_y, y_pred, average='weighted')
f1 = f1_score(test_y, y_pred, average='weighted')

print("Результаты для задачи классификации:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Precision: {precision:.4f}")
print(f"F1 Score: {f1:.4f}")


Результаты для задачи классификации:
Accuracy: 0.8429
Recall: 0.8429
Precision: 0.8434
F1 Score: 0.8415


### b. Обучить модели из sklearn (для регрессии) для выбранных наборов данных и оценить качество моделей (для регрессии) по выбранным метрикам на выбранных наборах данных

In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

data = pd.read_csv("CO2 Emission by Vehicles/CO2 Emissions_Canada.csv")

data = pd.get_dummies(data, columns=['Vehicle Class'], drop_first=True)

X = data[['Engine Size(L)', 'Cylinders', 'Fuel Consumption City (L/100 km)', 
          'Fuel Consumption Hwy (L/100 km)', 'Fuel Consumption Comb (L/100 km)'] + 
         [col for col in data.columns if 'Vehicle Class_' in col]]
y = data['CO2 Emissions(g/km)']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

dt_regressor = DecisionTreeRegressor(criterion='squared_error', max_depth=None, random_state=42)

dt_regressor.fit(X_train, y_train)

y_pred_test = dt_regressor.predict(X_test)

test_mae = mean_absolute_error(y_test, y_pred_test)
test_mse = mean_squared_error(y_test, y_pred_test)

print("Результаты для задачи регрессии:")
print(f"MAE: {test_mae:.2f}")
print(f"MSE: {test_mse:.2f}")

Результаты для задачи регрессии:
MAE: 2.80
MSE: 107.49


## 3. Улучшение бейзлайна

1. Формирование новых признаков
    - Комбинация признаков: Например, для задачи регрессии по CO2 выбросам можно создать новый признак, комбинирующий объем двигателя и расход топлива, или вывести дополнительные признаки, отражающие производительность.

2.  Подбор гиперпараметров
    - Для классификации:
        - criterion: выбор функции для оценки разбиений (gini, entropy).
        - max_depth: ограничение на глубину дерева.
        - min_samples_split: минимальное количество выборок для разбиения узла.
        - min_samples_leaf: минимальное количество выборок в листовом узле.
        - max_features: максимальное количество признаков, используемых для разбиения.
    - Для регрессии:
        - criterion: функция оценки качества разбиений (squared_error, absolute_error, poisson).
        - Остальные параметры аналогичны классификации.

3. Методы оптимизации
    - Grid Search: Провести поиск по сетке (Grid Search) для поиска оптимальных параметров модели.

In [12]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.model_selection import GridSearchCV

train_X, train_y = read_data('data/train.csv')
test_X, test_y = read_data('data/test.csv')

param_grid_classification = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 10, 20, 30, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': [None, 'sqrt', 'log2']
}

dt_classifier = DecisionTreeClassifier(random_state=42)
grid_search_classifier = GridSearchCV(estimator=dt_classifier, param_grid=param_grid_classification,
                                      scoring='f1_weighted', cv=5, verbose=2, n_jobs=-1)

grid_search_classifier.fit(train_X, train_y)

best_params_classifier = grid_search_classifier.best_params_
print("Лучшие параметры для классификации:", best_params_classifier)

best_classifier = grid_search_classifier.best_estimator_
y_pred = best_classifier.predict(test_X)

accuracy = accuracy_score(test_y, y_pred)
recall = recall_score(test_y, y_pred, average='weighted')
precision = precision_score(test_y, y_pred, average='weighted')
f1 = f1_score(test_y, y_pred, average='weighted')

print("Результаты для классификации с оптимальными параметрами:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Precision: {precision:.4f}")
print(f"F1 Score: {f1:.4f}")

Fitting 5 folds for each of 270 candidates, totalling 1350 fits
[CV] END criterion=gini, max_depth=None, max_features=None, min_samples_leaf=1, min_samples_split=2; total time=   3.5s
[CV] END criterion=gini, max_depth=None, max_features=None, min_samples_leaf=1, min_samples_split=5; total time=   3.6s
[CV] END criterion=gini, max_depth=None, max_features=None, min_samples_leaf=1, min_samples_split=2; total time=   3.9s
[CV] END criterion=gini, max_depth=None, max_features=None, min_samples_leaf=1, min_samples_split=2; total time=   3.8s
[CV] END criterion=gini, max_depth=None, max_features=None, min_samples_leaf=1, min_samples_split=5; total time=   3.8s
[CV] END criterion=gini, max_depth=None, max_features=None, min_samples_leaf=1, min_samples_split=2; total time=   3.9s
[CV] END criterion=gini, max_depth=None, max_features=None, min_samples_leaf=1, min_samples_split=2; total time=   4.1s
[CV] END criterion=gini, max_depth=None, max_features=None, min_samples_leaf=1, min_samples_spli

In [13]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import GridSearchCV

data = pd.read_csv("CO2 Emission by Vehicles/CO2 Emissions_Canada.csv")

data = pd.get_dummies(data, columns=['Vehicle Class'], drop_first=True)
X = data[['Engine Size(L)', 'Cylinders', 'Fuel Consumption City (L/100 km)', 
          'Fuel Consumption Hwy (L/100 km)', 'Fuel Consumption Comb (L/100 km)'] + 
         [col for col in data.columns if 'Vehicle Class_' in col]].values.astype(np.float64)
y = data['CO2 Emissions(g/km)'].values.astype(np.float64)

train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.3, random_state=42)

param_grid_regression = {
    'criterion': ['squared_error', 'absolute_error', 'poisson'],
    'max_depth': [None, 10, 20, 30, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': [None, 'sqrt', 'log2']
}

dt_regressor = DecisionTreeRegressor(random_state=42)
grid_search_regressor = GridSearchCV(estimator=dt_regressor, param_grid=param_grid_regression,
                                     scoring='neg_mean_squared_error', cv=5, verbose=2, n_jobs=-1)

grid_search_regressor.fit(train_X, train_y)

best_params_regressor = grid_search_regressor.best_params_
print("Лучшие параметры для регрессии:", best_params_regressor)

best_regressor = grid_search_regressor.best_estimator_
y_pred = best_regressor.predict(test_X)

mae = mean_absolute_error(test_y, y_pred)
mse = mean_squared_error(test_y, y_pred)

print("Результаты для регрессии с оптимальными параметрами:")
print(f"MAE: {mae:.4f}")
print(f"MSE: {mse:.4f}")


Fitting 5 folds for each of 405 candidates, totalling 2025 fits
[CV] END criterion=squared_error, max_depth=None, max_features=None, min_samples_leaf=1, min_samples_split=2; total time=   0.0s
[CV] END criterion=squared_error, max_depth=None, max_features=None, min_samples_leaf=1, min_samples_split=2; total time=   0.0s
[CV] END criterion=squared_error, max_depth=None, max_features=None, min_samples_leaf=1, min_samples_split=2; total time=   0.0s
[CV] END criterion=squared_error, max_depth=None, max_features=None, min_samples_leaf=1, min_samples_split=2; total time=   0.0s
[CV] END criterion=squared_error, max_depth=None, max_features=None, min_samples_leaf=1, min_samples_split=2; total time=   0.0s
[CV] END criterion=squared_error, max_depth=None, max_features=None, min_samples_leaf=1, min_samples_split=5; total time=   0.0s
[CV] END criterion=squared_error, max_depth=None, max_features=None, min_samples_leaf=1, min_samples_split=5; total time=   0.0s
[CV] END criterion=squared_error,

#### Добавление нового признака (для задачи регрессии)

In [14]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split

data = pd.read_csv("CO2 Emission by Vehicles/CO2 Emissions_Canada.csv")

data = pd.get_dummies(data, columns=['Vehicle Class'], drop_first=True)

X = data[['Engine Size(L)', 'Cylinders', 'Fuel Consumption City (L/100 km)', 
          'Fuel Consumption Hwy (L/100 km)', 'Fuel Consumption Comb (L/100 km)'] + 
         [col for col in data.columns if 'Vehicle Class_' in col]]
y = data['CO2 Emissions(g/km)']

X_train_new_field, X_test_new_field, y_train_new_field, y_test_new_field = train_test_split(X, y, test_size=0.3, random_state=42)

X_train_new_field['Engine_Fuel'] = X_train_new_field['Engine Size(L)'] * X_train_new_field['Fuel Consumption Comb (L/100 km)']
X_test_new_field['Engine_Fuel'] = X_test_new_field['Engine Size(L)'] * X_test_new_field['Fuel Consumption Comb (L/100 km)']

poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly.fit_transform(X_train_new_field)
X_test_poly = poly.transform(X_test_new_field)

dt_regressor = DecisionTreeRegressor(criterion='squared_error', max_depth=None, random_state=42)

dt_regressor.fit(X_train_poly, y_train_new_field)

y_pred_test = dt_regressor.predict(X_test_poly)

test_mae = mean_absolute_error(y_test_new_field, y_pred_test)
test_mse = mean_squared_error(y_test_new_field, y_pred_test)

print("Результаты для задачи регрессии:")
print(f"MAE: {test_mae:.2f}")
print(f"MSE: {test_mse:.2f}")


Результаты для задачи регрессии:
MAE: 2.52
MSE: 79.11


### Окончательный улучшенный бейзлайн

#### Задача классификации

Лучшие параметры для классификации: {'criterion': 'gini', 'max_depth': None, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 2}

In [22]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

train_X, train_y = read_data('data/train.csv')
test_X, test_y = read_data('data/test.csv')

dt_classifier = DecisionTreeClassifier(criterion='gini', max_depth=None,max_features=None, min_samples_leaf=1, min_samples_split=2, random_state=42)

dt_classifier.fit(train_X, train_y)

y_pred = dt_classifier.predict(test_X)

accuracy = accuracy_score(test_y, y_pred)
recall = recall_score(test_y, y_pred, average='weighted')
precision = precision_score(test_y, y_pred, average='weighted')
f1 = f1_score(test_y, y_pred, average='weighted')

print("Результаты для задачи классификации:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Precision: {precision:.4f}")
print(f"F1 Score: {f1:.4f}")


Результаты для задачи классификации:
Accuracy: 0.8626
Recall: 0.8626
Precision: 0.8636
F1 Score: 0.8620


#### Задача регрессии

Лучшие параметры для регрессии: {'criterion': 'squared_error', 'max_depth': 20, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 2}

In [4]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
import pandas as pd

data = pd.read_csv("CO2 Emission by Vehicles/CO2 Emissions_Canada.csv")

data = pd.get_dummies(data, columns=['Vehicle Class'], drop_first=True)

X = data[['Engine Size(L)', 'Cylinders', 'Fuel Consumption City (L/100 km)', 
          'Fuel Consumption Hwy (L/100 km)', 'Fuel Consumption Comb (L/100 km)'] + 
         [col for col in data.columns if 'Vehicle Class_' in col]]
y = data['CO2 Emissions(g/km)']

X_train_new_field, X_test_new_field, y_train_new_field, y_test_new_field = train_test_split(X, y, test_size=0.3, random_state=42)

X_train_new_field['Engine_Fuel'] = X_train_new_field['Engine Size(L)'] * X_train_new_field['Fuel Consumption Comb (L/100 km)']
X_test_new_field['Engine_Fuel'] = X_test_new_field['Engine Size(L)'] * X_test_new_field['Fuel Consumption Comb (L/100 km)']

poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly.fit_transform(X_train_new_field)
X_test_poly = poly.transform(X_test_new_field)

dt_regressor = DecisionTreeRegressor(criterion='squared_error', max_depth=20, max_features=None, min_samples_leaf=1, min_samples_split=2, random_state=42)

dt_regressor.fit(X_train_poly, y_train_new_field)

y_pred_test = dt_regressor.predict(X_test_poly)

test_mae = mean_absolute_error(y_test_new_field, y_pred_test)
test_mse = mean_squared_error(y_test_new_field, y_pred_test)

print("Результаты для задачи регрессии:")
print(f"MAE: {test_mae:.2f}")
print(f"MSE: {test_mse:.2f}")


Результаты для задачи регрессии:
MAE: 2.43
MSE: 64.43


### Выводы

После проведения оптимизации гиперпараметров и использования метода Grid Search для классификации, удалось добиться небольшого улучшения качества модели: метрика F1 Score выросли с 0.8415 до 0.8620, что свидетельствует о лучшей способности модели корректно классифицировать данные. Для задачи регрессии оптимизация гиперпараметров также привела к улучшению: метрика MAE снизилась с 2.80 до 2.43, а MSE — со 107.49 до 64.63, что указывает на уменьшение средних и квадратичных ошибок модели. 

Это подтверждает важность как правильной настройки модели, так и обогащения исходных данных.

## 4. Имплементация алгоритма машинного обучения 

### Базовый класс

In [23]:
class DecisionTreeBase:
    def __init__(self, max_depth=None, min_samples_split=2, min_samples_leaf=1):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.tree = None

    def _best_split(self, X, y):
        raise NotImplementedError("Этот метод должен быть реализован в дочернем классе.")

    def _build_tree(self, X, y, depth=0):
        if (self.max_depth is not None and depth >= self.max_depth) or len(y) < self.min_samples_split:
            return self._leaf_value(y)

        split = self._best_split(X, y)

        if split is None:
            return self._leaf_value(y)

        feature_index, threshold, left_mask, right_mask = split
        left_X, left_y = X[left_mask], y[left_mask]
        right_X, right_y = X[right_mask], y[right_mask]

        left_tree = self._build_tree(left_X, left_y, depth + 1)
        right_tree = self._build_tree(right_X, right_y, depth + 1)

        return {
            'feature_index': feature_index,
            'threshold': threshold,
            'left': left_tree,
            'right': right_tree
        }

    def fit(self, X, y):
        X = np.array(X)
        y = np.array(y)
        self.tree = self._build_tree(X, y)

    def _predict_sample(self, x, tree):
        if not isinstance(tree, dict):
            return tree

        feature_value = x[tree['feature_index']]
        if feature_value <= tree['threshold']:
            return self._predict_sample(x, tree['left'])
        else:
            return self._predict_sample(x, tree['right'])

    def predict(self, X):
        X = np.array(X)
        return np.array([self._predict_sample(row, self.tree) for row in X])


### Имплементация дерева решений для классификации

In [24]:
class DecisionTreeClassifierCustom(DecisionTreeBase):
    def __init__(self, max_depth=None, min_samples_split=2, min_samples_leaf=1):
        super().__init__(max_depth, min_samples_split, min_samples_leaf)

    def _gini_impurity(self, left_y, right_y):
        def gini(y):
            m = len(y)

            if m == 0:
                return 0
            
            p = np.bincount(y) / m

            return 1 - np.sum(p ** 2)

        total = len(left_y) + len(right_y)

        return (len(left_y) / total) * gini(left_y) + (len(right_y) / total) * gini(right_y)

    def _leaf_value(self, y):
        return np.bincount(y).argmax()

    def _best_split(self, X, y):
        best_gini = float('inf')
        best_split = None
        n_samples, n_features = X.shape

        for feature_index in range(n_features):
            thresholds = np.unique(X[:, feature_index])

            for threshold in thresholds:
                left_mask = X[:, feature_index] <= threshold
                right_mask = ~left_mask
                left_y, right_y = y[left_mask], y[right_mask]

                if len(left_y) >= self.min_samples_leaf and len(right_y) >= self.min_samples_leaf:
                    gini = self._gini_impurity(left_y, right_y)
                    if gini < best_gini:
                        best_gini = gini
                        best_split = (feature_index, threshold, left_mask, right_mask)

        return best_split

### Имплементация дерева решений для регрессии

In [25]:
class DecisionTreeRegressorCustom(DecisionTreeBase):
    def __init__(self, max_depth=None, min_samples_split=2, min_samples_leaf=1):
        super().__init__(max_depth, min_samples_split, min_samples_leaf)

    def _variance_reduction(self, left_y, right_y):
        def variance(y):
            return np.var(y) * len(y)

        total_len = len(left_y) + len(right_y)
        left_weight = len(left_y) / total_len
        right_weight = len(right_y) / total_len

        return variance(left_y) + variance(right_y)

    def _leaf_value(self, y):
        return np.mean(y)

    def _best_split(self, X, y):
        best_reduction = -float('inf')
        best_split = None
        n_samples, n_features = X.shape

        for feature_index in range(n_features):
            thresholds = np.unique(X[:, feature_index])

            for threshold in thresholds:
                left_mask = X[:, feature_index] <= threshold
                right_mask = ~left_mask
                left_y, right_y = y[left_mask], y[right_mask]

                if len(left_y) >= self.min_samples_leaf and len(right_y) >= self.min_samples_leaf:
                    reduction = self._variance_reduction(left_y, right_y)
                    if reduction > best_reduction:
                        best_reduction = reduction
                        best_split = (feature_index, threshold, left_mask, right_mask)

        return best_split


### Обучение моделей на выбранных датасетах и вывод метрик

#### Задача классификации

In [43]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def read_data(filepath):
    data = pd.read_csv(filepath)
    X = data.iloc[:, :-1]
    y = data.iloc[:, -1]
    return X, y

train_X, train_y = read_data('data/train.csv')
test_X, test_y = read_data('data/test.csv')

train_X = train_X.iloc[:800]
train_y = train_y.iloc[:800]

train_sample_indices = np.random.choice(train_X.index, size=800, replace=False)
train_X = train_X.loc[train_sample_indices].reset_index(drop=True)
train_y = train_y.loc[train_sample_indices].reset_index(drop=True)

label_encoder = LabelEncoder()
train_y = pd.Series(label_encoder.fit_transform(train_y), name=train_y.name)
test_y = pd.Series(label_encoder.transform(test_y), name=test_y.name)

dt_classifier = DecisionTreeClassifierCustom(max_depth=4, min_samples_split=2, min_samples_leaf=1)
dt_classifier.fit(train_X, train_y)

y_pred = dt_classifier.predict(test_X)

accuracy = accuracy_score(test_y, y_pred)
recall = recall_score(test_y, y_pred, average='weighted')
precision = precision_score(test_y, y_pred, average='weighted')
f1 = f1_score(test_y, y_pred, average='weighted')

print("Классификация:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Precision: {precision:.4f}")
print(f"F1 Score: {f1:.4f}")


Классификация:
Accuracy: 0.8575
Recall: 0.8575
Precision: 0.7821
F1 Score: 0.8082


/Users/daustinov/study/multimedia/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


#### Задача регрессии

In [55]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error

data = pd.read_csv("CO2 Emission by Vehicles/CO2 Emissions_Canada.csv")

data = pd.get_dummies(data, columns=['Vehicle Class'], drop_first=True)

X = data[['Engine Size(L)', 'Cylinders', 'Fuel Consumption City (L/100 km)', 
          'Fuel Consumption Hwy (L/100 km)', 'Fuel Consumption Comb (L/100 km)'] + 
         [col for col in data.columns if 'Vehicle Class_' in col]]
y = data['CO2 Emissions(g/km)']

scaler = StandardScaler()
train_X_scaled = scaler.fit_transform(train_X)
test_X_scaled = scaler.transform(test_X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

dt_regressor = DecisionTreeRegressorCustom(max_depth=300)
dt_regressor.fit(X_train, y_train)
y_pred = dt_regressor.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print("Регрессия:")
print(f"MAE: {mae:.2f}")
print(f"MSE: {mse:.2f}")


Регрессия:
MAE: 7.05
MSE: 251.37


### Сравнение результатов с п.2. Выводы

Кастомная модель показала хороший результат, но проигрывает модели sklearn по всем метрикам, особенно в показателе Precision (0.7821 против 0.8636). Это может свидетельствовать о том, что кастомная модель допускает больше ложных срабатываний (false positives), в то время как модель sklearn более сбалансированно обрабатывает данные.

Разница в метриках между кастомной и библиотечной моделями для регрессии оказалась значительно больше, чем в задаче классификации. Модель sklearn существенно точнее, демонстрируя минимальные ошибки (MAE и MSE). Это говорит о том, что кастомная модель пока плохо справляется с задачей регрессии и требует оптимизации гиперпараметров. 

### Улучшение бейзлайна

#### Добавление нового признака (для задачи регрессии)

In [54]:
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split

data = pd.read_csv("CO2 Emission by Vehicles/CO2 Emissions_Canada.csv")

data = pd.get_dummies(data, columns=['Vehicle Class'], drop_first=True)

X = data[['Engine Size(L)', 'Cylinders', 'Fuel Consumption City (L/100 km)', 
          'Fuel Consumption Hwy (L/100 km)', 'Fuel Consumption Comb (L/100 km)'] + 
         [col for col in data.columns if 'Vehicle Class_' in col]]
y = data['CO2 Emissions(g/km)']

X_train_new_field, X_test_new_field, y_train_new_field, y_test_new_field = train_test_split(X, y, test_size=0.3, random_state=42)

X_train_new_field['Engine_Fuel'] = X_train_new_field['Engine Size(L)'] * X_train_new_field['Fuel Consumption Comb (L/100 km)']
X_test_new_field['Engine_Fuel'] = X_test_new_field['Engine Size(L)'] * X_test_new_field['Fuel Consumption Comb (L/100 km)']

poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly.fit_transform(X_train_new_field)
X_test_poly = poly.transform(X_test_new_field)

dt_regressor = DecisionTreeRegressorCustom(max_depth=300)
dt_regressor.fit(X_train, y_train)
y_pred = dt_regressor.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print("Регрессия:")
print(f"MAE: {mae:.2f}")
print(f"MSE: {mse:.2f}")

Регрессия:
MAE: 7.05
MSE: 251.37


#### Оптимизация гиперпараметров и Grid Search

##### Задача классификации

In [46]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import KFold
from itertools import product

def read_data(filepath):
    data = pd.read_csv(filepath)
    X = data.iloc[:, :-1]
    y = data.iloc[:, -1]
    return X, y

def grid_search_classification(model_class, param_grid, X, y, metrics):
    best_score = -float('inf')
    best_params = None
    best_metrics = None
    
    keys, values = zip(*param_grid.items())
    param_combinations = [dict(zip(keys, v)) for v in product(*values)]
    
    for params in param_combinations:
        model = model_class(**params)
        scores = []
        fold_metrics = []
        
        kf = KFold(n_splits=5, shuffle=True, random_state=42)
        for train_idx, val_idx in kf.split(X):
            X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
            y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
            
            model.fit(X_train, y_train)
            y_pred = model.predict(X_val)
            fold_metrics.append({name: metric(y_val, y_pred) for name, metric in metrics.items()})
            scores.append(metrics['accuracy'](y_val, y_pred))
        
        avg_score = np.mean(scores)
        avg_metrics = {name: np.mean([fold[name] for fold in fold_metrics]) for name in metrics}
        
        if avg_score > best_score:
            best_score = avg_score
            best_params = params
            best_metrics = avg_metrics
    
    return best_params, best_metrics

train_X, train_y = read_data('data/train.csv')
test_X, test_y = read_data('data/test.csv')

train_X = train_X.iloc[:800]
train_y = train_y.iloc[:800]

train_sample_indices = np.random.choice(train_X.index, size=800, replace=False)
train_X = train_X.loc[train_sample_indices].reset_index(drop=True)
train_y = train_y.loc[train_sample_indices].reset_index(drop=True)

label_encoder = LabelEncoder()
train_y = pd.Series(label_encoder.fit_transform(train_y), name=train_y.name)
test_y = pd.Series(label_encoder.transform(test_y), name=test_y.name)

param_grid = {
    'max_depth': [3, 5],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

metrics = {
    'accuracy': accuracy_score,
    'recall': lambda y_true, y_pred: recall_score(y_true, y_pred, average='weighted'),
    'precision': lambda y_true, y_pred: precision_score(y_true, y_pred, average='weighted'),
    'f1': lambda y_true, y_pred: f1_score(y_true, y_pred, average='weighted')
}

best_params, best_metrics = grid_search_classification(
    model_class=DecisionTreeClassifierCustom,
    param_grid=param_grid,
    X=train_X,
    y=train_y,
    metrics=metrics
)

print("Лучшие параметры для классификации:", best_params)
print("Средние метрики на кросс-валидации:", best_metrics)

dt_classifier = DecisionTreeClassifierCustom(**best_params)
dt_classifier.fit(train_X, train_y)

y_pred = dt_classifier.predict(test_X)
test_metrics = {name: metric(test_y, y_pred) for name, metric in metrics.items()}

print("Тестовые метрики:")
for name, value in test_metrics.items():
    print(f"{name.capitalize()}: {value:.4f}")


/Users/daustinov/study/multimedia/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/daustinov/study/multimedia/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/daustinov/study/multimedia/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()

Лучшие параметры для классификации: {'max_depth': 5, 'min_samples_split': 2, 'min_samples_leaf': 1}
Средние метрики на кросс-валидации: {'accuracy': np.float64(0.9949999999999999), 'recall': np.float64(0.9949999999999999), 'precision': np.float64(0.9952165120507512), 'f1': np.float64(0.9949976722623781)}
Тестовые метрики:
Accuracy: 0.9104
Recall: 0.9104
Precision: 0.9136
F1: 0.9102


##### Задача регрессии

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split

def grid_search_regression(model_class, param_grid, X, y, metrics):
    best_score = float('inf')
    best_params = None
    best_metrics = None
    
    keys, values = zip(*param_grid.items())
    param_combinations = [dict(zip(keys, v)) for v in product(*values)]
    
    for params in param_combinations:
        model = model_class(**params)
        scores = []
        fold_metrics = []
        
        kf = KFold(n_splits=5, shuffle=True, random_state=42)
        for train_idx, val_idx in kf.split(X):
            X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
            y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
            
            model.fit(X_train, y_train)
            y_pred = model.predict(X_val)
            fold_metrics.append({name: metric(y_val, y_pred) for name, metric in metrics.items()})
            scores.append(metrics['mse'](y_val, y_pred))
        
        avg_score = np.mean(scores)
        avg_metrics = {name: np.mean([fold[name] for fold in fold_metrics]) for name in metrics}
        
        if avg_score < best_score:
            best_score = avg_score
            best_params = params
            best_metrics = avg_metrics
    
    return best_params, best_metrics

data = pd.read_csv("CO2 Emission by Vehicles/CO2 Emissions_Canada.csv")
data = pd.get_dummies(data, columns=['Vehicle Class'], drop_first=True)

X = data[['Engine Size(L)', 'Cylinders', 'Fuel Consumption City (L/100 km)', 
          'Fuel Consumption Hwy (L/100 km)', 'Fuel Consumption Comb (L/100 km)'] + 
         [col for col in data.columns if 'Vehicle Class_' in col]]
y = data['CO2 Emissions(g/km)']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

param_grid = {
    'max_depth': [100, 300, 500],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

metrics = {
    'mae': mean_absolute_error,
    'mse': mean_squared_error
}

best_params, best_metrics = grid_search_regression(
    model_class=DecisionTreeRegressorCustom,
    param_grid=param_grid,
    X=X_train,
    y=y_train,
    metrics=metrics
)

print("Лучшие параметры для регрессии:", best_params)
print("Средние метрики на кросс-валидации:", best_metrics)

dt_regressor = DecisionTreeRegressorCustom(**best_params)
dt_regressor.fit(X_train, y_train)

y_pred = dt_regressor.predict(X_test)
test_metrics = {name: metric(y_test, y_pred) for name, metric in metrics.items()}

print("Тестовые метрики:")
for name, value in test_metrics.items():
    print(f"{name.upper()}: {value:.2f}")


Лучшие параметры для регрессии: {'max_depth': 500, 'min_samples_split': 2, 'min_samples_leaf': 1}
Средние метрики на кросс-валидации: {'mae': np.float64(4.521109864894752), 'mse': np.float64(152.73647327842505)}
Тестовые метрики:
MAE: 4.16
MSE: 172.70


### Окончательный улучшенный бейзлайн

#### Задача классификации

Лучшие параметры для классификации: {'max_depth': 5, 'min_samples_split': 2, 'min_samples_leaf': 1}

In [45]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def read_data(filepath):
    data = pd.read_csv(filepath)
    X = data.iloc[:, :-1]
    y = data.iloc[:, -1]
    return X, y

train_X, train_y = read_data('data/train.csv')
test_X, test_y = read_data('data/test.csv')

train_X = train_X.iloc[:800]
train_y = train_y.iloc[:800]

train_sample_indices = np.random.choice(train_X.index, size=800, replace=False)
train_X = train_X.loc[train_sample_indices].reset_index(drop=True)
train_y = train_y.loc[train_sample_indices].reset_index(drop=True)

label_encoder = LabelEncoder()
train_y = pd.Series(label_encoder.fit_transform(train_y), name=train_y.name)
test_y = pd.Series(label_encoder.transform(test_y), name=test_y.name)

dt_classifier = DecisionTreeClassifierCustom(max_depth=5, min_samples_split=2, min_samples_leaf=1)
dt_classifier.fit(train_X, train_y)

y_pred = dt_classifier.predict(test_X)

accuracy = accuracy_score(test_y, y_pred)
recall = recall_score(test_y, y_pred, average='weighted')
precision = precision_score(test_y, y_pred, average='weighted')
f1 = f1_score(test_y, y_pred, average='weighted')

print("Классификация:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Precision: {precision:.4f}")
print(f"F1 Score: {f1:.4f}")

Классификация:
Accuracy: 0.9104
Recall: 0.9104
Precision: 0.9136
F1 Score: 0.9102


#### Задача регрессии

Лучшие параметры для регрессии: {'max_depth': 500, 'min_samples_split': 2, 'min_samples_leaf': 1}

In [53]:
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split

data = pd.read_csv("CO2 Emission by Vehicles/CO2 Emissions_Canada.csv")

data = pd.get_dummies(data, columns=['Vehicle Class'], drop_first=True)

X = data[['Engine Size(L)', 'Cylinders', 'Fuel Consumption City (L/100 km)', 
          'Fuel Consumption Hwy (L/100 km)', 'Fuel Consumption Comb (L/100 km)'] + 
         [col for col in data.columns if 'Vehicle Class_' in col]]
y = data['CO2 Emissions(g/km)']

X_train_new_field, X_test_new_field, y_train_new_field, y_test_new_field = train_test_split(X, y, test_size=0.3, random_state=42)

X_train_new_field['Engine_Fuel'] = X_train_new_field['Engine Size(L)'] * X_train_new_field['Fuel Consumption Comb (L/100 km)']
X_test_new_field['Engine_Fuel'] = X_test_new_field['Engine Size(L)'] * X_test_new_field['Fuel Consumption Comb (L/100 km)']

poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly.fit_transform(X_train_new_field)
X_test_poly = poly.transform(X_test_new_field)

dt_regressor = DecisionTreeRegressorCustom(max_depth=500, min_samples_leaf=1, min_samples_split=2)
dt_regressor.fit(X_train, y_train)
y_pred = dt_regressor.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print("Регрессия:")
print(f"MAE: {mae:.2f}")
print(f"MSE: {mse:.2f}")

Регрессия:
MAE: 4.16
MSE: 172.70


### Выводы

После проведения улучшений в бейзлайне для самописных моделей удалось значительно повысить их качество, особенно для задачи классификации. На начальном этапе базовые метрики классификатора показывали достаточно удовлетворительные результаты, такие как Accuracy, Recall и F1 Score на уровне 0.8568, однако Precision составил всего 0.7820, что указывало на значительное количество ложноположительных предсказаний. В задаче регрессии изначальные значения ошибок MAE (45.47) и MSE (3265.88) свидетельствовали о высокой погрешности модели, что говорило о необходимости доработки.

Добавление нового признака для задачи регрессии не повлияло на качество модели, что подтвердили неизменные значения MAE и MSE. Это указывает на то, что данный признак не нес дополнительной информации для улучшения предсказаний. Однако значительное улучшение качества моделей было достигнуто на этапе подбора гиперпараметров с использованием Grid Search. Для задачи классификации удалось добиться хороших: Accuracy, Recall, Precision и F1 Score достигли значений около 0.91. В задаче регрессии также наблюдалось значительное улучшение: MAE снизился до 4.16, а MSE до 172.70, что свидетельствует о повышении точности предсказаний модели.

Если сравнивать метрики самописных моделей и моделей из sklearn, то значительное увеличение точности было достигнуто на задаче классификации, по сравнению с моделью из sklearn. В задаче регрессии самописная модель показала метрики хуже, чем у модели из sklearn.

Таким образом, оптимизация гиперпараметров оказалась наиболее значимым шагом в улучшении качества моделей, что подтверждается резким ростом метрик классификации и заметным снижением ошибок в регрессии.